## VST Demo

This notebook showcases `vst` version `0.1.1` with updated functions and syntax to demonstrate a basic but complete model analysis workflow, using a simple toy model:
1. Partial model calibrations using variant model files (generation of these is manual for now but could easily be automated)
2. Full model calibration, building on the results of previous partial calibrations
3. Calibration with MCMC to derive a sample of posterior parameter estimates
4. Semi-automated generation of additional files to run projections
5. Basic projections + parametric sensitivity (for uncertainty intervals) based on calibrated parameter estimates
6. Projections for policy analysis + sensitivity analysis

At the moment this notebook relies on simple import of `vst.py` (and by extension `vst_text.py`); proper module packaging will occur eventually. For documentation on the `vst` code, objects, and functions, see the `.py` file or `.ipynb` notebooks.

**NOTE:** this version of `vst` is *not* backward-compatible with existing scripts: 1) `Script` objects and use syntax have been substantially modified, and 2) newest Vengine no longer has the expiration popup to bypass, so `press` lines are no longer needed and have been removed.

#### 0. Set up necessary files
Before starting, ensure that all files needed for this demo (included in the `VST Testing` folder) are in the same directory, along with this `ipynb` file. The demo will create multiple output and other files, so it is best to use a clean directory.

##### 0.1 Import all `VST` classes and functions
The `from <module> import *` syntax imports functions directly (without prefix); it is not generally recommended but is used here for simplicity.

In [ ]:
from vst import *

##### 0.2 Set up control file
Each `Script` object reads in a `controlfile` from which it derives most of its command script settings. Technically, each `Script` instance can use a different `controlfile`, but in most cases `controlfile` fields are shared across most of any given model analysis workflow, so it is useful to define a 'primary' version and modify it as needed later in the workflow. This can be done inline at the head of a `ipynb` notebook, or separately to be read in as a JSON file.

For details on each of these arguments, see the main `vst` notebook documentation.

**NOTE:** Be sure to change `vgpath` and `vspath` as needed for your setup to avoid errors!

In [ ]:
# Global control inputs
timelimit = 60  # Time limit for automatic checking/monitoring of runs
vgpath = "C:/Program Files/Vengine/Vengine.exe"  # Change this to your Vengine file path!
vspath = "C:/Program Files/Vensim/vendss64.exe"  # Change this to your Vensim file path!
samplefrac = 0.05  # Fraction of MCMC sample to use for sensitivity analyses


# Controlfile inputs
cf = {
    'basename': 'Test', 
    'simcontrol': {
        "model": "prey-predator.mdl", 
        "data": [
            "hudson-bay-lynx-hare.vdf"
            ], 
        "payoff": "prey-predator.vpd", 
        "sensitivity": "", 
        "optparm": "prey-predator.voc", 
        "changes": [], 
        "savelist": "", 
        "senssavelist": "PrPrSave.lst"
    }, 
    'runcmd': '', 
    'savecmd': ''    
}

# Create logfile
log = f"{os.getcwd()}/log.txt"

#### 1. Run partial model calibrations
`vst` is designed for easy setup of different workflows, such as hierarchical model estimation or iterative estimation of partial models.

We demonstrate here using a simple pair of 'partial' model calibrations, using *only* prey-side *or* predator-side data to estimate the corresponding parameters, before combining the outputs of those partial model calibrations as a starting point for calibrating the full model. (Obviously, starting with partial calibrations is unnecessary for this simple model, but is done here for demonstration purposes.)

##### 1.1 Partial model calibration of prey-side parameters
Basic `Script` use syntax is:
```
x = Script(controlfile, name, logfile, sfxs=suffixes, chglist=changes)
x.compile_script(logfile, **kwargs)
```
This will create an instance `x` of a `Script` object with the specified settings (detailed in `vst` documentation), compile it into a `cmd` file, and execute that `cmd` file in Vensim/Vengine to produce a Vensim run and output. 

By default, the `cmd` file and standard Vensim optimisation outputs, e.g. `vdf[x]` / `out` / `rep` files, should show up in your working directory; here the `subdir='Hare'` argument in the second line will instead place all working files in the specified subdirectory, copying only the `out` file to the main working directory. (This is useful for controlling file proliferation.) 

Note the use of the `sfxs` argument to specify alternate versions (designated with `_h` suffix) of the `voc` and `vpd` files. For this demo, these files have been created manually ahead of time; in a full automated workflow, they could be procedurally generated from base versions of those files.

In [ ]:
# Start with partial model calibration of just prey-side parameters
prey = Script(cf, "hare", log, sfxs={'optparm': '_h', 'payoff': '_h'})
prey.compile_script(vgpath, log, subdir='Hare')  # Run this in subdirectory 'Hare'

##### 1.2 Partial model calibration of predator-side parameters
Similar to the previous step, this will run a partial model calibration in a new subdirectory 'Lynx', using the alternate `_l` versions of `voc` and `vpd` files. Additionally, the `chglist` argument will read in the `out` file from the previous step (named `<basename>hare.out`) as an input into the calibration.

(The exact syntax for `chglist` is powerful and flexible but somewhat complicated; don't worry about it for now, or read the documentation for details.)

In [ ]:
# Continue with partial model calibration of just predator-side parameters
pred = Script(cf, "lynx", log, sfxs={'optparm': '_l', 'payoff': '_l'}, chglist=[('hare', '')])
pred.compile_script(vgpath, log, subdir='Lynx')  # Run this in subdirectory 'Lynx'

#### 2. Run full model calibration
Next we run a calibration of the full model in another subdirectory 'Full', this time using `chglist` to read in both previous outputs (`<basename>hare.out` and `<basename>lynx.out`) as a starting point for the calibration. This pattern of reading in previous outputs is useful for speeding up convergence in an iterative calibration workflow.

In [ ]:
# Then do a calibration of the full model with all params
# Note use of chglist to read in previous .out files
both = Script(cf, "fullopt", log, chglist=[(['hare', 'lynx'], '')])
both.compile_script(vgpath, log, subdir='Full')

#### 3. Run MCMC to get posterior parameter sample
Once we have Powell optimisation results for parameter estimates (specifically, the `<basename>fullopt.out` file), we can feed those in to an MCMC estimation to get a sample of the joint posterior parameter distribution. You *could* run the MCMC estimation directly, especially for simple models, but for larger models it is usually much more computationally efficient to start the MCMC estimation from an already calibrated parameter set.

The `downsample` function is used to take a random subsample of the MCMC posterior parameter sample to use in sensitivity analyses; the full MCMC sample will typically be far too large. It also creates a `vsc` file for sensitivity analysis to read in that subsample.

In [ ]:
# Then run MCMC, reading in full optimisation results
mcmc = Script(cf, "fullMC", log, sfxs={'optparm': '_m'}, chglist=[('full', 'opt')])
mcmc.compile_script(vgpath, log, subdir='Full')

mcmc.downsample(samplefrac)  # Downsample MCMC result to get input for sensitivity

#### 4. Create additional files to run projections
Making projections based on estimated parameters is a very common use case for a model. While exploratory analysis of projections can be done manually, it is often useful to automate the process. Doing so often requires supplemental files such as `cin` files to alter certain model characteristics (e.g. time bounds, switches, inputs), which can themselves be generated in an automated way.

For projections to include uncertainty, we first need to create a modified version of the main `controlfile` with appropriate settings for sensitivity analysis (`senscf`). We also dynamically create a `cin` file that extends the time horizon of the model (in this case, by 10 time units). Doing this dynamically makes the workflow robust to e.g. future changes in the `FINAL TIME` of the model, which will make your life easier as your model evolves.

In [ ]:
# Set up alternate control file for sensitivity runs
senscf = cf.copy()  # Make a true separate copy
senscf['simcontrol']['sensitivity'] = f"{mcmc.runname}.vsc"  # Specify sensitivity control file
senscf['savecmd'] = 'SENS2FILE|!|!|T'  # Save full sensitivity output in Tidy format

# Create CIN file for projections, extending final time by 10
finaltime = get_value(cf['simcontrol']['model'], 'FINAL TIME')  # Dynamically read in final time
with open('Projection.cin', 'w') as f:
    f.write(f"FINAL TIME = {finaltime + 10}")  # Extend by 10 units for projections

#### 5. Run basic projections & parametric sensitivity analysis
When making projections, you will almost always want to start with a simple base case using the estimated parameters and no other changes. (Sometimes you may want multiple base cases with differing assumptions about key inputs.) Doing so provides a baseline against which effects of other changes can be compared.

We do two runs here - a simple model run (as indicated by `simtype='r'`), followed by a sensitivity run (`simtype='s'`) that uses the previously derived MCMC subsample to generate uncertainty / credible intervals on the projections. We include `Projection.cin`, created in the previous step, in the `chglist`, in order to change the `FINAL TIME` of the model for these runs for projecting future model behaviour.

Note that we use Vensim rather than Vengine for the simple model run (specifying `vengine=False` and using `vspath` instead of `vgpath`); for simple model runs, Vensim can often be faster, both because it's less buggy and because you can turn off compilation (which may not be worth the computational overhead for a simple run).

We make several changes for the sensitivity run:
- We use the previously defined `senscf` to create the `Script` instance; the modified control file has the necessary `cmd` commands for running sensitivity analysis instead of optimisation.
- We also change `timelimit` from its global value of 60 to 300, as exporting sensitivity output can take a while and 60 seconds may be too little time. `vst` monitors Vengine closely to ensure nothing has bugged out, including checking periodically for activity (see main documentation for details). Setting `timelimit` too short can cause it to get trapped in an infinite loop if it thinks Vengine has bugged out when in reality it's legitimately just taking a while, e.g. for the large-file sensitivity export.
- We specify different `check_funcs`, which are used to monitor Vengine for buggy output (see main documentation for details). The default `check_funcs` are intended for monitoring optimisation runs; `check_output` is appropriate for sensitivity runs.

Note that this step will create some large exported `tab` files containing the full sensitivity analysis output (all 5000 runs) in the 'Projections' subdirectory.

In [ ]:
# Do a simple projection using estimated parameters
proj = Script(cf, "projbase", log, chglist=[('full', 'opt'), 'Projection.cin'], simtype='r')
# Note use of Vensim rather than Vengine for simple run; in practice could use either
proj.compile_script(vspath, log, subdir='Projections', vengine=False)

# Then a sensitivity analysis using previous MCMC subsample
sens = Script(senscf, "sensbase", log, chglist=[('full', 'opt'), 'Projection.cin'], simtype='s')
sens.compile_script(vgpath, log, subdir='Projections', timelimit=300, check_funcs=[check_output])

#### 6. Run policy analysis w/ parametric sensitivity
Finally, we can test the impact of different policies or scenarios on projected behaviour by making projections with additional model changes (e.g. `cin` files, such as `Hunting.cin` incorporated in the `chglist` here) to represent the effects of those policies. The outcomes (i.e. resultant projected behaviour) can then be compared against baseline projections to quantify the potential impacts of policies.

`vst` does not currently include tools for post-processing and analysis of model outputs, but those are in the works!

In [ ]:
# Then read in a 'Hunting' policy scenario and project effects
hunt = Script(cf, "projhunt", log, chglist=[('full', 'opt'), 'Projection.cin', 'Hunting.cin'], simtype='r')
hunt.compile_script(vspath, log, subdir='Projections', vengine=False)

# And conduct sensitivity analysis on 'Hunting' scenario with MCMC subsample
shunt = Script(senscf, "senshunt", log, chglist=[('full', 'opt'), 'Projection.cin', 'Hunting.cin'], simtype='s')
shunt.compile_script(vgpath, log, subdir='Projections', timelimit=300, check_funcs=[check_output])

In [ ]:
000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000